# IMDB 
- lexicon

In [69]:
import numpy as np
import pandas as pd

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
df=pd.read_csv('/content/drive/MyDrive/0422B/labeledTrainData.tsv',sep='\t',quoting=3) #3:quote_none. 따옴표 같은 거 제대로 들어가게.
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


### Wordnet Synset 및 sentiwordnet SentiSysnet 클래스

In [66]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.corpus import wordnet

In [75]:
from nltk.corpus.reader.wordnet import Synset

term = 'fly'
synsets = wordnet.synsets(term)

In [74]:
type(synsets), len(synsets)

(list, 20)

In [76]:
for synset in synsets:
    print(f'##### name:{synset.name()} #####')
    print('POS:',synset.lexname())
    print('정의:',synset.definition())
    print('표제어:',synset.lemma_names())

##### name:fly.n.01 #####
POS: noun.animal
정의: two-winged insects characterized by active flight
표제어: ['fly']
##### name:tent-fly.n.01 #####
POS: noun.artifact
정의: flap consisting of a piece of canvas that can be drawn back to provide entrance to a tent
표제어: ['tent-fly', 'rainfly', 'fly_sheet', 'fly', 'tent_flap']
##### name:fly.n.03 #####
POS: noun.artifact
정의: an opening in a garment that is closed by a zipper or by buttons concealed under a fold of cloth
표제어: ['fly', 'fly_front']
##### name:fly.n.04 #####
POS: noun.act
정의: (baseball) a hit that flies up in the air
표제어: ['fly', 'fly_ball']
##### name:fly.n.05 #####
POS: noun.artifact
정의: fisherman's lure consisting of a fishhook decorated to look like an insect
표제어: ['fly']
##### name:fly.v.01 #####
POS: verb.motion
정의: travel through the air; be airborne
표제어: ['fly', 'wing']
##### name:fly.v.02 #####
POS: verb.motion
정의: move quickly or suddenly
표제어: ['fly']
##### name:fly.v.03 #####
POS: verb.motion
정의: operate an airplane
표제어: ['f

- 어휘 간의 유사도

In [77]:
# 단어, 품사를 모를 경우에는 synsets()으로 알아냄
for synset in wordnet.synsets('tiger'):
    print(synset.name(), synset.definition())

tiger.n.01 a fierce or audacious person
tiger.n.02 large feline of forests in most of Asia having a tawny coat with black stripes; endangered


In [78]:
# 단어, 품사를 아는 경우에는 synset() 
tiger = wordnet.synset('tiger.n.02')
tree = wordnet.synset('tree.n.01')
lion = wordnet.synset('lion.n.01')
cat = wordnet.synset('cat.n.01')
dog = wordnet.synset('dog.n.01')

In [79]:
# 단어간의 유사도
tiger.path_similarity(lion), tiger.path_similarity(dog), tiger.path_similarity(tree)

(0.3333333333333333, 0.16666666666666666, 0.07142857142857142)

In [80]:
# 5개 단어간의 유사도
similarities = []
entities = [tree, lion, tiger, cat, dog]
for entity in entities:
    similarity = [entity.path_similarity(another) for another in entities]
    similarities.append(similarity)

In [81]:
df = pd.DataFrame(similarities, columns=['tree','lion','tiger','cat','dog'],
                  index=['tree','lion','tiger','cat','dog'])
df

,tree,lion,tiger,cat,dog
tree,1.000000,0.071429,0.071429,0.076923,0.125000
lion,0.071429,1.000000,0.333333,0.250000,0.166667
tiger,0.071429,0.333333,1.000000,0.250000,0.166667
cat,0.076923,0.250000,0.250000,1.000000,0.200000
dog,0.125000,0.166667,0.166667,0.200000,1.000000


- SentiSynset 클래스


In [15]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [16]:
from nltk.corpus import sentiwordnet
senti_synsets = list(sentiwordnet.senti_synsets('slow'))

In [17]:
senti_synsets

[SentiSynset('decelerate.v.01'),
 SentiSynset('slow.v.02'),
 SentiSynset('slow.v.03'),
 SentiSynset('slow.a.01'),
 SentiSynset('slow.a.02'),
 SentiSynset('dense.s.04'),
 SentiSynset('slow.a.04'),
 SentiSynset('boring.s.01'),
 SentiSynset('dull.s.08'),
 SentiSynset('slowly.r.01'),
 SentiSynset('behind.r.03')]

In [20]:
#father 단어의 긍정/부정/객관성 지수
father=sentiwordnet.senti_synset('father.n.01')
father.pos_score(),father.neg_score(),father.obj_score()

(0.0, 0.0, 1.0)

In [22]:
#fabulous 단어의 긍정/부정/객관성 지수
fabulous=sentiwordnet.senti_synset('fabulous.a.01')
fabulous.pos_score(),fabulous.neg_score(),fabulous.obj_score()

(0.875, 0.125, 0.0)

In [23]:
wordnet.NOUN, wordnet.ADJ, wordnet.ADV,wordnet.VERB

('n', 'a', 'r', 'v')

- 감성지수 계산

In [27]:
from nltk import word_tokenize, pos_tag
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
sentence="It's good to see you again."
word_list=word_tokenize(sentence)
word_list

['It', "'s", 'good', 'to', 'see', 'you', 'again', '.']

In [31]:
nltk.download('averaged_perceptron_tagger')
pos_tag(word_list)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('It', 'PRP'),
 ("'s", 'VBZ'),
 ('good', 'JJ'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('you', 'PRP'),
 ('again', 'RB'),
 ('.', '.')]

In [33]:
def penn_to_wordnet(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('R'):
        return wordnet.ADV
    if tag.startswith('V'):
        return wordnet.VERB

In [34]:
for word, pos in pos_tag(word_list):
    print(word, penn_to_wordnet(pos))

It None
's v
good a
to None
see v
you None
again r
. None


- Sentence로부터 감성지수를 계산하는 과정

In [35]:
sentence="It's good to see you again."
word_list=[word for word in word_tokenize(sentence) if len(word) > 2]
word_list

['good', 'see', 'you', 'again']

In [37]:
for word, pos in pos_tag(word_list):
    wn_tag=penn_to_wordnet(pos)
    if wn_tag: #none이 아닌 4가지 형태만 가져온다.
        synsets=list(sentiwordnet.senti_synsets(word,wn_tag))
        synset=synsets[0]
        print(synset)

<good.a.01: PosScore=0.75 NegScore=0.0>
<see.n.01: PosScore=0.0 NegScore=0.0>
<again.r.01: PosScore=0.0 NegScore=0.0>


In [39]:
sentiment=0
for word, pos in pos_tag(word_list):
    wn_tag=penn_to_wordnet(pos)
    if wn_tag: #none이 아닌 4가지 형태만 가져온다.
        synsets=list(sentiwordnet.senti_synsets(word,wn_tag))
        synset=synsets[0]
        sentiment +=synset.pos_score() - synset.neg_score()
print(sentiment)

0.75


- 표제어 추출도 하기.

In [40]:
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [42]:
sentiment=0
for word, pos in pos_tag(word_list):
    wn_tag=penn_to_wordnet(pos)
    if wn_tag: #none이 아닌 4가지 형태만 가져온다.
        lemma=lemmatizer.lemmatize(word,wn_tag)
        synsets=list(sentiwordnet.senti_synsets(lemma,wn_tag))
        synset=synsets[0]
        sentiment +=synset.pos_score() - synset.neg_score()
print(sentiment)

0.75


- 도큐먼트에서 감성지수를 계산하는 과정 및 함수

In [43]:
from nltk import sent_tokenize
document= '''
Worries about yet another Batman coming into the fray should be alleviated very quickly as director Matt Reeves version of the caped crusader done by the way of Seven takes hold early on and refuses too let go on its way to a well earned three hour runtime that is full of surprises, atmosphere, historical nods and artistry as Reeve's David Fincher like epic takes us on a memorable Gotham adventure that may divide casual fans, all the while becoming a new favourite for long term fans of DC Comics most famous creation.
'''

In [44]:
sentiment=0
for sentence in sent_tokenize(document):
    word_list=[word for word in word_tokenize(sentence) if len(word) > 2]
    for word, pos in pos_tag(word_list):
        wn_tag=penn_to_wordnet(pos)
        if wn_tag: #none이 아닌 4가지 형태만 가져온다.
            lemma=lemmatizer.lemmatize(word,wn_tag)
            synsets=list(sentiwordnet.senti_synsets(lemma,wn_tag))
            if not synsets:
                print(word)
                continue
            synset=synsets[0]
            sentiment +=synset.pos_score() - synset.neg_score()
print('긍정' if sentiment >= 0 else "부정")

caped
runtime
atmosphere
Fincher
Gotham
긍정


- 함수 만들기

In [50]:
def swn_polarity(text):
    lemmatizer = WordNetLemmatizer()
    sentiment=0
    for sentence in sent_tokenize(text):
        word_list=[word for word in word_tokenize(sentence) if len(word) > 2]
        for word, pos in pos_tag(word_list):
            wn_tag=penn_to_wordnet(pos)
            if wn_tag: #none이 아닌 4가지 형태만 가져온다.
                lemma=lemmatizer.lemmatize(word,wn_tag)
                synsets=list(sentiwordnet.senti_synsets(lemma,wn_tag))
                if not synsets:
                    continue
                synset=synsets[0]
                sentiment +=synset.pos_score() - synset.neg_score()

    return 1 if sentiment >=0 else 0

- IMDB 영화평 감성분석

In [51]:
#<br />태그는 공백으로 바꾸기.
df.review = df.review.str.replace('<br />',' ')
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
df=df.head(10000)

In [53]:
%time df['pred']=df.review.apply(lambda x: swn_polarity(x))

CPU times: user 3min 35s, sys: 1.57 s, total: 3min 37s
Wall time: 3min 40s


In [54]:
#정확도 계산
from sklearn.metrics import accuracy_score
accuracy_score(df.sentiment, df.pred)

0.6309

## VADER Lexicon을 이용한 감성 분석
- 코드도 간결하고 속도도 빠르고 정확도도 높다!


In [57]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [58]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti_analyzer = SentimentIntensityAnalyzer()
senti_analyzer.polarity_scores(df.review[0])

{'compound': -0.7943, 'neg': 0.13, 'neu': 0.743, 'pos': 0.127}

In [59]:
def vader_polarity(document,threshold=0.1):
    score=senti_analyzer.polarity_scores(document)
    return 1 if score['compound'] >= threshold else 0


In [60]:
%time df['vader'] = df.review.apply(lambda x:vader_polarity(x,0.1))

CPU times: user 32.6 s, sys: 257 ms, total: 32.8 s
Wall time: 33.2 s


In [61]:
accuracy_score(df.sentiment, df.vader)

0.6997